In [1]:
import warnings
warnings.filterwarnings("ignore")
%matplotlib inline
import matplotlib.pyplot as plt
import csv
import pandas as pd
import sklearn
import numpy as np
import time
import datetime
import re
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC, LinearSVC
from sklearn.metrics import classification_report, f1_score, accuracy_score, confusion_matrix
from sklearn.pipeline import Pipeline
from sklearn.grid_search import GridSearchCV
from sklearn.cross_validation import StratifiedKFold, cross_val_score, train_test_split 
from textblob import TextBlob
from wordcloud import WordCloud
from sklearn.cluster import KMeans
from sklearn.preprocessing import LabelEncoder
import matplotlib
from matplotlib import pyplot as plt
matplotlib.style.use("ggplot")

df = pd.read_csv("data/train.csv")
df.head()

/home/ayrus/anaconda3/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
/home/ayrus/anaconda3/lib/python3.6/site-packages/sklearn/grid_search.py:42: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)


ln       dob gn       fn
0      SMITH JR  01/03/68  F  WILLIAM
1  ROTHMEYER JR  01/03/68  F  WILLIAM
2       ASBY JR  01/03/68  F  WILLIAM
3     SALTER JR  01/03/68  F  WILLIAM
4     SALTER JR  01/03/68  F  WILLIAM

In [2]:
df['fn_len'] = df['fn'].apply(lambda val:len(val))

In [3]:
df.head()

ln       dob gn       fn  fn_len
0      SMITH JR  01/03/68  F  WILLIAM       7
1  ROTHMEYER JR  01/03/68  F  WILLIAM       7
2       ASBY JR  01/03/68  F  WILLIAM       7
3     SALTER JR  01/03/68  F  WILLIAM       7
4     SALTER JR  01/03/68  F  WILLIAM       7

In [4]:
df.sort_values('fn_len',ascending=False,inplace=True)
df.reset_index(inplace=True)
df.drop(['index'],1,inplace=True)
df.head(6)

ln       dob gn            fn  fn_len
0        HANNA  24/11/34  M  ADDISON JOHN      12
1        HANNA  24/11/34  M     ADDISON J       9
2   GRIFFIN JR  07/05/37  M      CLARENCE       8
3  HOUGHTON JR  31/01/46  M      LAWRENCE       8
4  HOUGHTON JR  31/01/46  M      LAWRENCE       8
5      LIND JR  31/01/46  M      LAWRENCE       8

In [5]:
j = 0
df['fn_label'] = df['fn_len']
df['fn_label'][0] = 0
for i in range(1,len(df)):
    if(df['fn'][i] in df['fn'][i-1]):
        df['fn_label'][i] = df['fn_label'][i-1]
    else:
        j += 1
        df['fn_label'][i] = j

In [6]:
df.head(5)

ln       dob gn            fn  fn_len  fn_label
0        HANNA  24/11/34  M  ADDISON JOHN      12         0
1        HANNA  24/11/34  M     ADDISON J       9         0
2   GRIFFIN JR  07/05/37  M      CLARENCE       8         1
3  HOUGHTON JR  31/01/46  M      LAWRENCE       8         2
4  HOUGHTON JR  31/01/46  M      LAWRENCE       8         2

In [7]:
df['ln_len'] = df['ln'].apply(lambda val:len(val))
df.sort_values('ln_len',ascending=False,inplace=True)
df.reset_index(inplace=True)
df.drop(['index'],1,inplace=True)

j = 0
df['ln_label'] = df['ln_len']
df['ln_label'][0] = 0
for i in range(1,len(df)):
    if(df['ln'][i] in df['ln'][i-1]):
        df['ln_label'][i] = df['ln_label'][i-1]
    else:
        j += 1
        df['ln_label'][i] = j

In [8]:
df.head(5)

ln       dob gn    fn  fn_len  fn_label  ln_len  ln_label
0  MICHAELSON JR  24/06/39  M  JOHN       4        31      13         0
1  MICHAELSON JR  16/12/44  M  JOHN       4        31      13         0
2  MICHAELSON JR  17/06/48  M   ROY       3        33      13         0
3  MICHAELSON JR  24/06/39  M  JACK       4        32      13         0
4  MICHAELSON JR  24/06/39  M  JOHN       4        31      13         0

In [9]:
df["val_gn"] = df["gn"].map({"M":0,"F":1})

In [10]:
def DOB_to_unix(str):
    s = str.split('/')
    str = s[0] + '/' + s[1]+ '/19' + s[2]
    
    return int((time.mktime(datetime.datetime.strptime(str, "%d/%m/%Y").timetuple())))

df["val_dob"] = df["dob"].apply(lambda x: DOB_to_unix(x))

In [11]:
df.head()

ln       dob gn    fn  fn_len  fn_label  ln_len  ln_label  \
0  MICHAELSON JR  24/06/39  M  JOHN       4        31      13         0   
1  MICHAELSON JR  16/12/44  M  JOHN       4        31      13         0   
2  MICHAELSON JR  17/06/48  M   ROY       3        33      13         0   
3  MICHAELSON JR  24/06/39  M  JACK       4        32      13         0   
4  MICHAELSON JR  24/06/39  M  JOHN       4        31      13         0   

   val_gn    val_dob  
0       0 -963293400  
1       0 -790324200  
2       0 -679815000  
3       0 -963293400  
4       0 -963293400

In [12]:
train = df[['val_dob','val_gn','fn_label','ln_label']]
n_cluster = len(train)
kmn = KMeans(n_clusters=n_cluster)
kmn.fit(train)
groups = kmn.predict(train)
df['type'] = groups
print(len(pd.DataFrame(groups)[0].unique()))

60


In [19]:
df.sort_values('type',inplace=True)
df = df.drop_duplicates(subset=['type'])

In [36]:
df.reset_index(inplace=True)
df.drop(['index'],1,inplace=True)
df.head()
df = df[['ln','dob','gn','fn']]

In [38]:
df.to_csv('type.csv',index=None)